# 爬取网络小说

In [ ]:
# 爬取多本的小说
import re
import requests
import os
from bs4 import BeautifulSoup
from lxml import etree
#url ='https://book.qidian.com/info/1015207338#Catalog'
#请求头
#爬取多本小说
global clist
global ann
#url ='https://book.qidian.com/info/1015207338#Catalog'
header={'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
url1='https://www.qidian.com/all?orderId=&page=1&vip=0&style=1&pageSize=20&siteid=1&pubflag=0&hiddenField=0'
#获取小说名以及小说的链接
def getbooklist(url1):
    books=requests.get(url1,headers=header).text
    objects=etree.HTML(books)
    #<div class="book-mid-info">
    objs=objects.xpath('//div[@class="book-mid-info"]/h4')
    ann=[]
    for obj in objs:
        try:
            book_urls=obj.xpath('a/@href')[0]
            print(book_urls)
        #print(obj)
        #获取小说名称,不属于属性
            book_names=obj.xpath('a/text()')[0]
            print(book_names)
        #字典
            into1 ={
                  'book_urls':'https:'+book_urls,
                  'book_names':book_names
            }
            ann.append(into1)
        except:
            pass
    #clist=[]
    #clist.append(into)
    return ann
    #text=' '
    #for j in content1:
        #text=text+j
#global clist
#url ='https://book.qidian.com/info/1015207338#Catalog'
#header={'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
#获取章节名以及章节的链接
def getbookurls(url):
    #获取页面源代码
    charptes=requests.get(url,headers=header).text
    objects=etree.HTML(charptes)
    #//匹配所有
    objs=objects.xpath('//ul[@class="cf"]/li')
    #print(objs)
    #用try,except检查错误
    clist=[]
    for obj in objs:
        try:
            chapt_urls=obj.xpath('a/@href')[0]
            print(chapt_urls)
        #print(obj)
        #获取小说名称,不属于属性
            chapt_names=obj.xpath('a/text()')[0]
            print(chapt_names)
        #字典
            into ={
                  'chapt_urls':'https:'+chapt_urls,
                  'chapt_names':chapt_names
            }
            clist.append(into)
        except:
            pass
    #clist=[]
    #clist.append(into)
    return clist
#clist=getbookurls(url)
#print(clist)
#获取小说具体内容
def getcontent(url):
    #获取数据
    res=requests.get(url,headers=header).text
    objects=etree.HTML(res)
    #解析内容
    objs=objects.xpath('//div[@class="read-content j_readContent"]/p/text()')
    content=[]
    for i in objs:
        print(i)
        text=i.replace('\u3000\u3000',' ')
        content.append(text)
    return content

    #clist=getbookurls()
ann=getbooklist(url1)
num=0
for i in ann:
    book_urls=i['book_urls']
    book_name=i['book_names']
    print("正在下载 %s"%book_name)
    clist=getbookurls(book_urls)
    num=num+1
    #(school_name+'.data').decode('utf-8')
    #with open(r'E:\python\小说'+str(num)+'\%s'%book_name , 'w')as f:
       # f.write(book_name)
    path="E:\\python\\noval"+str(num)+str(book_name)
    #新建文件夹
    if not os.path.exists(path):
    #path="E:\\python\\noval"+str(num)+str(book_name)
            os.mkdir(path, 755)
            #print("正在下载 %s"%book_name)
            for i in clist:
                chapt_urls=i['chapt_urls']
                chapt_name=i['chapt_names']
                content=getcontent(chapt_urls)
                text=' '
                for j in content:
                    text=text+j
                    #写入文件
                    with open(r'E:\\python\\noval'+str(num)+str(book_name)+'\\%s'%chapt_name,'w')as f:
                        try:
                            f.write(text)
                        except:
                            pass
    else:
        pass
    #text=' '
    #for j in content1:
        #text=text+j
    #print("正在下载 %s"%book_name)
    time_sleep = 100
    #print("正在下载 %s"%chapt_name)
    

# 词频分析，关键词抽取

In [ ]:
import jieba
import os
from jieba import analyse
# 引入TF-IDF关键词抽取接口
tfidf = analyse.extract_tags
rootdir0 = r'E:\python'
list0 = os.listdir(rootdir0) #列出文件夹下所有的目录与文件
print(list0)
for j in range(0,len(list0)):
    data={}
    path = os.path.join(rootdir0,list0[j])
    if os.path.isdir(path):
        print(path)
        li = os.listdir(path) 
        for i in range(0,len(li)):
            path1 = os.path.join(path,li[i])
            if os.path.isfile(path1):
                fb=open(path1,'r')
                content=fb.read()
                fb.close()
        #words=jieba.lcut(content)
                keywords = tfidf(content)
                for word in keywords:
                    if len(word)>1:#去掉单个字符
                        if word in data:
                            data[word]+=1
                        else:
                            data[word]=1
        hist= list(data.items())#转成列表，便于排序
        hist.sort(key=lambda x:x[1],reverse=True)
#排序
        for i in range(50):
            print('{:<10}{:>5}'.format(hist[i][0],hist[i][1]))
#print(hist)

# 情感分析

In [ ]:
#coding:UTF-8
import jieba
import sys
import os
from snownlp import SnowNLP
from snownlp import sentiment
input_file=r'E:\python\noval1从1983开始'
#output_file='E:/result.txt/'
def read_and_analysis(input_file):
    #f = open(r'E:\python\noval1从1983开始)         
    rootdir = r'E:\python\noval1从1983开始'
    list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
    #print(list)
    for j in range(0,len(list)):
    #data={}
        path = os.path.join(rootdir,list[j])
        #print(path)
        if os.path.isfile(path):
            #print(path)
            fb=open(path,'r')
            while True:
                line = fb.readline()
                #print(line)
                if not line:
                        break
                lines = line.strip().split("\t")
                #print(lines)
                #if len(lines) < 2:
                   #print(len(lines))
                    #continue
                s= SnowNLP(lines[0])
    # s.words 查询分词结果
                seg_words = ""
        
                print(s.keywords(50))
                for x in s.words:
                    seg_words += "_"
                    seg_words += x
                print(s.sentiments)
    print(s.sentiments)
    fw.close()

if __name__ == "__main__":
    # 重新训练模型
    sentiment.train('E:/result/neg.txt', 'E:/result/pos.txt')
  # 保存好新训练的模型
    sentiment.save('sentiment.marshal')
    read_and_analysis(input_file)

In [ ]:
#coding:utf-8
require 'rest-client'
require 'json'

def send_file(*argv)
    link = 'http://api.bosonnlp.com/sentiment/analysis'
    # 填写注册得到的API Token
    headers = {
        'X-Token'=>'xxxxx',
        'Content-Type'=>'application/json',
        'Accept'=>'application/json'
    }
    a =[]
     argv.each do |x|
        s = File.read(x)
        data = s.to_json
        resp = RestClient.post(link,data,headers)
        v = resp.gsub("[","").gsub("]","").split(',')
        res = {"file_name"=>x,"optimism"=>v[0].strip,"pessimistic"=>v[1].strip}
        a<< res
     end
     puts a
end
send_file("t.txt","1.txt")